In [13]:
import sys
sys.path.append("/Users/abcom/Desktop/github/reviewturtl")
from reviewturtl.src.agents.summarizer_agent import SummarizerAgent
from reviewturtl.src.agents.code_search_agent import CodeSearchAgent

In [14]:
from dotenv import load_dotenv
load_dotenv(dotenv_path="/Users/abcom/Desktop/github/reviewturtl/.env")

True

In [2]:
from reviewturtl.settings import initialize_dspy_with_configs
initialize_dspy_with_configs()

In [3]:
import dspy

In [19]:
agents = [SummarizerAgent, CodeSearchAgent]
agents_dict = {}
arg_agent_mapper = {}
for agent in agents:
    agent_init = agent()
    agents_dict[agent_init.class_name] = agent_init.desc
    arg_agent_mapper[agent_init.class_name] = agent_init.input_variables

In [20]:
agents_dict

{'SummarizerAgent': 'A summarizer agent that summarizes the changes in a file diff.',
 'CodeSearchAgent': 'A code search agent that searches for code in a repository.'}

In [9]:
from typing import List, Dict
class ReactSignature(dspy.Signature):
    __doc__ = """
    INSTRUCTIONS TO FOLLOW\n
    >>> Available agents are the agents that are available to answer the query. They are in a dictionary format with the key as the agent name and the value as the agent description\n
    >>> Think step by step on what agents are best suited to answer the query\n
    >>> Choose the best agents (can be more than one) to answer the query\n
    >>> Return the agents in a list\n
    """
    conversation_history: str = dspy.InputField(desc="The conversation history so far")
    query: str = dspy.InputField(
        desc="The latest user query based on which the agent is to be chosen"
    )
    available_agents: Dict[str, str] = dspy.InputField(
        desc="The Dictionary of available agents with the agent name as the key and the agent description as the value"
    )
    agents_to_use: List[str] = dspy.OutputField(desc="The List of agents to use")

In [34]:
pg = dspy.TypedChainOfThought(ReactSignature)
pred = pg(conversation_history="", query="i need to implement health endpoint for the api", available_agents=agents_dict)

In [35]:
pred

Prediction(
    reasoning='{\n  "value": [\n    "CodeSearchAgent"\n  ]\n}',
    agents_to_use=['CodeSearchAgent']
)

In [36]:
agent_being_used = pred.agents_to_use[0]
args_to_find = arg_agent_mapper[agent_being_used]

In [37]:
args_to_find

['search_query', 'context_related_to_search_query']

In [ ]:
agent_being_used

In [15]:
file_diff = """
diff --git a/reviewturtl/src/agents/summarizer_agent.py b/reviewturtl/src/agents/summarizer_agent.py
index 83b2c9d..f3b2c9d 100644
--- a/reviewturtl/src/agents/summarizer_agent.py
+++ b/reviewturtl/src/agents/summarizer_agent.py
@@ -1,6 +1,7 @@
 from reviewturtl.src.programmes.typed_cot import DspyProgramme
 from reviewturtl.src.signatures.signatures import SummarizerSignature
 from reviewturtl.src.agents.base_agent import Agent
+import logging
 
 
 class SummarizerAgent(Agent):
@@ -8,6 +9,8 @@ class SummarizerAgent(Agent):
 
     def forward(self, file_diff):
         self.prediction_object = self.programme.predictor(file_diff=file_diff)
+        logging.info("Prediction object created")
         return self.prediction_object
 
     def walkthrough(self):
@@ -20,6 +23,7 @@ class SummarizerAgent(Agent):
 
     def __call__(self, file_diff):
         return self.forward(file_diff)
+        logging.info("SummarizerAgent called with file_diff")
 
 
 __all__ = ["SummarizerAgent"]

"""

In [28]:
class FindArgumentsForAgent(dspy.Signature):
    __doc__ = """
    INSTRUCTIONS TO FOLLOW\n
    >>> 
    """
    context: Dict[str, str] = dspy.InputField(desc="The context whatever is provided to be used for finding the arguments")
    agent_being_used_with_desc: Dict[str, str] = dspy.InputField(desc="The agent being used with the agent name and the agent description")
    argumets_to_find: List[str] = dspy.InputField(desc="The arguments to find for the agent")
    args_with_values: Dict[str, str] = dspy.OutputField(desc="The mapping of the arguments to the agent with key as the argument name and value as the argument value")

In [4]:
args_to_find

NameError: name 'args_to_find' is not defined

In [5]:
pg = dspy.TypedChainOfThought(FindArgumentsForAgent)
pred = pg(context={'file_diff': file_diff,'commit_hash':"2034","query":"i want to summarize the changes in this file"}, agent_being_used_with_desc={agents_dict[agent_being_used]:agent_being_used}, argumets_to_find=args_to_find)
pred

NameError: name 'dspy' is not defined

In [16]:
from reviewturtl.src.agents.utils import agent_dict
agent_dict

{'SummarizerAgent': {'class_name': 'SummarizerAgent',
  'description': 'A summarizer agent that summarizes the changes in a file diff.',
  'input_variables': ['file_diff'],
  'class_object': <reviewturtl.src.agents.summarizer_agent.SummarizerAgent at 0x3013caf90>},
 'CodeSearchAgent': {'class_name': 'CodeSearchAgent',
  'description': 'A code search agent that searches for code in a repository.',
  'input_variables': ['search_query', 'collection_name'],
  'class_object': <reviewturtl.src.agents.code_search_agent.CodeSearchAgent at 0x3044a2210>}}

In [17]:
list(agent_dict.keys())

['SummarizerAgent', 'CodeSearchAgent']

In [18]:
ob = agent_dict["SummarizerAgent"]["class_object"]

In [19]:
from reviewturtl.src.agents.react_agent import ReactAgent
ragent = ReactAgent()
context = {"file_diff":file_diff,"collection_name":"code_search"}
answer = ragent(conversation_history="", query="please tell me how to implement health endpoint for the api", context=context)

2024-08-17 22:42:03,556 - reviewturtl.src.agents.react_agent - INFO - Provided Agents: {'SummarizerAgent': 'A summarizer agent that summarizes the changes in a file diff.', 'CodeSearchAgent': 'A code search agent that searches for code in a repository.'}
2024-08-17 22:42:03,569 - reviewturtl.src.agents.react_agent - INFO - Agents to use: ['CodeSearchAgent']
2024-08-17 22:42:03,570 - reviewturtl.src.agents.react_agent - INFO - Agent Being Used With Desc: {'CodeSearchAgent': 'A code search agent that searches for code in a repository.'}
2024-08-17 22:42:03,570 - reviewturtl.src.agents.react_agent - INFO - Args to find: ['search_query', 'collection_name']
2024-08-17 22:42:03,581 - reviewturtl.src.agents.react_agent - INFO - Agent Caller: {'CodeSearchAgent': {'search_query': 'SummarizerAgent', 'collection_name': 'code_search'}}
2024-08-17 22:42:03,581 - reviewturtl.src.agents.react_agent - INFO - Args for agent: {'search_query': 'SummarizerAgent', 'collection_name': 'code_search'}


Batches: 100%|██████████| 1/1 [00:00<00:00,  2.33it/s]
2024-08-17 22:42:04,097 - httpx - INFO - HTTP Request: POST http://localhost:6333/collections/code_search/points/query "HTTP/1.1 200 OK"
2024-08-17 22:42:13,800 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-08-17 22:42:19,740 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [20]:
print(answer)

To implement a health endpoint for your API, you can follow these general steps:

1. **Define the Health Endpoint**: Decide on the URL path for your health endpoint, commonly `/health` or `/status`.

2. **Create the Endpoint Handler**: Implement a function that will handle requests to the health endpoint. This function should check the health of your application and return an appropriate response.

3. **Check Application Health**: In the handler function, perform checks to ensure that your application is running correctly. This might include checking database connections, external service availability, or other critical components.

4. **Return Health Status**: Return a response indicating the health status of your application. This is typically a JSON object with a status field.

Here is an example implementation in Python using the Flask framework:

```python
from flask import Flask, jsonify

app = Flask(__name__)

@app.route('/health', methods=['GET'])
def health_check():
    # Perf